In [6]:
# Import libs
import numpy as np
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

import requests 


### Define Foursquare Credentials and Version

In [24]:
CLIENT_ID = 'CRYPL3E3LQZ2W2TQO5MXB4VKFKMMNKRW1BORN3UNLCMMOHOT'
CLIENT_SECRET = 'VM41FNBBEBYZUZKAK413IMIYRXQZKWNOIDFTO5EMUMNE4NXR'
VERSION = '20180604'

### Categories of Toronto Venues

Extracting the categories of venues in Toronto to compare it with Dubai venues.

In [100]:
address = 'Toronto, Ontario, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_lng = location.longitude

url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}&ll={},{}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            toronto_lat, 
            toronto_lng)

toronto_cat_df = requests.get(url).json()

toronto_cat_list = pd.json_normalize(toronto_cat_df['response']['categories'])

len(toronto_cat_list)

toronto_cat = pd.DataFrame([])

i = 0
for cat in toronto_cat_list['name']:
    df_temp = pd.json_normalize(toronto_cat_df['response']['categories'][i]['categories'])
    df_temp['Category'] = cat
    toronto_cat = toronto_cat.append([df_temp])
    i = i +1
    

len(toronto_cat)

toronto_cat = toronto_cat[['id','Category','name', 'pluralName', 'shortName']]
toronto_cat.reset_index(drop=True, inplace=True)

toronto_cat.to_csv('Toronto Full Categories.csv')


### Categories of Dubai Venues

Extracting categories of in Dubai.

In [101]:
address = 'Dubai, UAE'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
dubai_lat = location.latitude
dubai_lng = location.longitude
# print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}&ll={},{}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude)

dubai_cat_df = requests.get(url).json()

Dubai_cat_list = pd.json_normalize(dubai_cat_df['response']['categories'])

len(toronto_cat_list)

dubai_cat = pd.DataFrame([])

i = 0
for cat in Dubai_cat_list['name']:
    df_temp = pd.json_normalize(dubai_cat_df['response']['categories'][i]['categories'])
    df_temp['Category'] = cat
    dubai_cat = dubai_cat.append([df_temp])
    i = i +1
    

len(dubai_cat)

dubai_cat = dubai_cat[['id','Category','name', 'pluralName', 'shortName']]
dubai_cat.reset_index(drop=True, inplace=True)

dubai_cat.to_csv('Dubai Full Categories.csv')

### Venues of Dubai

Now we will extract the venues of Dubai in Jumeirah Area.

In [123]:
radius = 20000
limit = 100 

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            dubai_lat, 
            dubai_lng,
#             toronto_lat, 
#             toronto_lng,
            radius,
            limit
            )

dubai_ven_df = requests.get(url).json()
len(dubai_ven_df['response']['venues'])

dubai_ven_normalized = pd.json_normalize(dubai_ven_df['response']['venues'])

dubai_ven_normalized = dubai_ven_normalized[["name", "categories", "location.address", "location.lat", "location.lng"]]
dubai_ven_normalized.rename(columns = {"name":"Name", "location.address": "Address", "location.lat": "Latitude", "location.lng": "Longitude"}, inplace = True)

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dubai_ven_normalized['categories'] = dubai_ven_normalized.apply(get_category_type, axis=1)
dubai_ven_normalized.rename(columns = {"categories":"Categories"}, inplace = True)

print(dubai_ven_normalized.shape)
dubai_ven_normalized.head()


(51, 5)


,Name,Categories,Address,Latitude,Longitude
0,Springs 6,Park,NaN,25.068680,55.188343
1,Mahiti,Nightclub,Jumeira Beach Hotel,25.073670,55.190872
2,Media Rotana Hotel,Hotel,Al Barsha South - TECOM,25.099642,55.179447
3,Han Cuisine,Chinese Restaurant,NaN,25.065233,55.193030
4,Xenia DXB,Mediterranean Restaurant,Spring Souq,25.066483,55.191847


### Explore Location in the Dubai Map

In [125]:

# create map
map_dubai_venues = folium.Map(location=[dubai_lat,dubai_lng], zoom_start=13)

# add markers to the map
markers_colors = []
for lat, lng, label in zip(dubai_ven_normalized['Latitude'], dubai_ven_normalized['Longitude'], dubai_ven_normalized['Name']):
    labels = folium.Popup(str(label), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=labels,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7).add_to(map_dubai_venues)
       
map_dubai_venues